In [9]:
import pandas as pd   #importing panda
import numpy as np    #importing numpy

In [10]:
df=pd.read_csv("train.csv",usecols=["Id","Ins_Age","BMI","Medical_History_1","Response"])   
#importing the dataset "train.csv" of Prudential Life Insurance Assesment Challenge and storing it in df
#only took the 5 columns of ID,Age,BMI,Medical History 1, Response; not the entire dataset

In [ ]:
df_temp=pd.read_csv("train.csv",nrows=1)

Index(['Id', 'Product_Info_1', 'Product_Info_2', 'Product_Info_3',
       'Product_Info_4', 'Product_Info_5', 'Product_Info_6', 'Product_Info_7',
       'Ins_Age', 'Ht',
       ...
       'Medical_Keyword_40', 'Medical_Keyword_41', 'Medical_Keyword_42',
       'Medical_Keyword_43', 'Medical_Keyword_44', 'Medical_Keyword_45',
       'Medical_Keyword_46', 'Medical_Keyword_47', 'Medical_Keyword_48',
       'Response'],
      dtype='object', length=128)


In [12]:
df.head()

,Id,Ins_Age,BMI,Medical_History_1,Response
0,2,0.641791,0.323008,4.0,8
1,5,0.059701,0.272288,5.0,4
2,6,0.029851,0.428780,10.0,8
3,7,0.164179,0.352438,0.0,8
4,8,0.417910,0.424046,NaN,8


In [13]:
df.isnull().sum()    #calcuating the null entries in the dataset

Id                      0
Ins_Age                 0
BMI                     0
Medical_History_1    8889
Response                0
dtype: int64

In [16]:
df.describe()    #various aspects of the columns in the dataset like mean, median, min, max etc.

,Id,Ins_Age,BMI,Medical_History_1,Response
count,59381.000000,59381.000000,59381.000000,50492.000000,59381.000000
mean,39507.211515,0.405567,0.469462,7.962172,5.636837
std,22815.883089,0.197190,0.122213,13.027697,2.456833
min,2.000000,0.000000,0.000000,0.000000,1.000000
25%,19780.000000,0.238806,0.385517,2.000000,4.000000
50%,39487.000000,0.402985,0.451349,4.000000,6.000000
75%,59211.000000,0.567164,0.532858,9.000000,8.000000
max,79146.000000,1.000000,1.000000,240.000000,8.000000


In [17]:
df.median()   #calculating median of each column

Id                   39487.000000
Ins_Age                  0.402985
BMI                      0.451349
Medical_History_1        4.000000
Response                 6.000000
dtype: float64

In [18]:
df["Medical_History_1"].fillna(4,inplace=True)  
#for completing the data, the entries which were Nan in the column 'Medical_History_1' were filled with the median
#of the same column.
#as the data is positively skewed, the median is used instead of mean for filling the Nan entries
#because of the fact that the mean was on the lower end of the graph and median was near the middle  

In [20]:
df.isnull().sum()   #confirming that no entries are Nan after filling the missing entries

Id                   0
Ins_Age              0
BMI                  0
Medical_History_1    0
Response             0
dtype: int64

In [21]:
random_days=np.random.randint(low=0,high=2000,size=len(df))    #generating a random sample equal to the length of df

In [22]:
start_date=pd.to_datetime("2010-01-01")     #assuming a starting date for calculation
df["Issue_Date"]=start_date + pd.to_timedelta(random_days,unit="D")    #creating a column of issue date of the policy by adding the random sample of days created from 0 to 2000 (converting them into dates) and adding them to the start date

In [23]:
risk_probs = 0.01 + df["Ins_Age"] * 0.15  # Base risk 1% + Age Factor
df["Claim_Flag"] = np.random.binomial(n=1, p=risk_probs)   #generating claim flag using random binomial with base risk and age factor

In [25]:
df["delay_days"]=np.random.randint(low=15,high=1825,size=len(df))     #generating a random sample of time between 15 days and 1 year

In [26]:
df["Payment_Date"]=df["Issue_Date"] + pd.to_timedelta(df["delay_days"],unit="D")     #generating a random date of payment of claim(if made) by converting delay days into time and adding it to issue date

In [28]:
#for the purpose of creating chainladder and triangle of claim payment we need the year of issue, payment, and time between them both
df["Origin_Year"]=df["Issue_Date"].dt.year    #extracting year from the issue date and storing it in 'Origin Year'
df.head()

,Id,Ins_Age,BMI,Medical_History_1,Response,Issue_Date,Claim_Flag,delay_days,Payment_Date,Origin_Year
0,2,0.641791,0.323008,4.0,8,2012-02-03,0,1598,2016-06-19,2012
1,5,0.059701,0.272288,5.0,4,2012-12-04,0,819,2015-03-03,2012
2,6,0.029851,0.428780,10.0,8,2010-04-03,0,818,2012-06-29,2010
3,7,0.164179,0.352438,0.0,8,2011-09-18,0,1197,2014-12-28,2011
4,8,0.417910,0.424046,4.0,8,2011-09-20,0,501,2013-02-02,2011


In [29]:
df["Development_Year"]=df["Payment_Date"].dt.year    #extracting the year from Payment date and storing it in Development_Year
df.head()

,Id,Ins_Age,BMI,Medical_History_1,Response,Issue_Date,Claim_Flag,delay_days,Payment_Date,Origin_Year,Development_Year
0,2,0.641791,0.323008,4.0,8,2012-02-03,0,1598,2016-06-19,2012,2016
1,5,0.059701,0.272288,5.0,4,2012-12-04,0,819,2015-03-03,2012,2015
2,6,0.029851,0.428780,10.0,8,2010-04-03,0,818,2012-06-29,2010,2012
3,7,0.164179,0.352438,0.0,8,2011-09-18,0,1197,2014-12-28,2011,2014
4,8,0.417910,0.424046,4.0,8,2011-09-20,0,501,2013-02-02,2011,2013


In [30]:
df["Development_Lag"]=df["Development_Year"] - df["Origin_Year"]    #Calculating the time lag between Payment year and Issue year and storing it in Development_Lag
df.head()

,Id,Ins_Age,BMI,Medical_History_1,Response,Issue_Date,Claim_Flag,delay_days,Payment_Date,Origin_Year,Development_Year,Development_Lag
0,2,0.641791,0.323008,4.0,8,2012-02-03,0,1598,2016-06-19,2012,2016,4
1,5,0.059701,0.272288,5.0,4,2012-12-04,0,819,2015-03-03,2012,2015,3
2,6,0.029851,0.428780,10.0,8,2010-04-03,0,818,2012-06-29,2010,2012,2
3,7,0.164179,0.352438,0.0,8,2011-09-18,0,1197,2014-12-28,2011,2014,3
4,8,0.417910,0.424046,4.0,8,2011-09-20,0,501,2013-02-02,2011,2013,2


In [31]:
df["Sum_Assured"]=np.random.randint(low=50000,high=500000,size=len(df))   #generating and assigning random Sum_Assured to everyone in the dataset between the amount of 50000 and 500000
df.head()

,Id,Ins_Age,BMI,Medical_History_1,Response,Issue_Date,Claim_Flag,delay_days,Payment_Date,Origin_Year,Development_Year,Development_Lag,Sum_Assured
0,2,0.641791,0.323008,4.0,8,2012-02-03,0,1598,2016-06-19,2012,2016,4,131174
1,5,0.059701,0.272288,5.0,4,2012-12-04,0,819,2015-03-03,2012,2015,3,445968
2,6,0.029851,0.428780,10.0,8,2010-04-03,0,818,2012-06-29,2010,2012,2,313457
3,7,0.164179,0.352438,0.0,8,2011-09-18,0,1197,2014-12-28,2011,2014,3,387053
4,8,0.417910,0.424046,4.0,8,2011-09-20,0,501,2013-02-02,2011,2013,2,258678


In [32]:
df["Claim_Amount"]=df["Claim_Flag"]*df["Sum_Assured"]    #Calculating the claim_amount if the person has Claim_Flag of 1 and multiplying it with the Sum_Assured
#in this way, if claim_Flag is 0 i.e. if no claim is made, then Sum_Assured becomes 0
df.head()

,Id,Ins_Age,BMI,Medical_History_1,Response,Issue_Date,Claim_Flag,delay_days,Payment_Date,Origin_Year,Development_Year,Development_Lag,Sum_Assured,Claim_Amount
0,2,0.641791,0.323008,4.0,8,2012-02-03,0,1598,2016-06-19,2012,2016,4,131174,0
1,5,0.059701,0.272288,5.0,4,2012-12-04,0,819,2015-03-03,2012,2015,3,445968,0
2,6,0.029851,0.428780,10.0,8,2010-04-03,0,818,2012-06-29,2010,2012,2,313457,0
3,7,0.164179,0.352438,0.0,8,2011-09-18,0,1197,2014-12-28,2011,2014,3,387053,0
4,8,0.417910,0.424046,4.0,8,2011-09-20,0,501,2013-02-02,2011,2013,2,258678,0


In [33]:
triangle=df.pivot_table(index="Origin_Year",columns="Development_Lag",values="Claim_Amount",aggfunc="sum")     #creating a claims triangle
print(triangle)

Development_Lag         0         1         2         3         4         5
Origin_Year                                                                
2010             21163791  46577974  47387821  42667790  48871079  17799882
2011             18474512  42617620  44252425  44604041  41985411  27468717
2012             20757711  39711852  46916956  44523032  50109365  20836835
2013             23638975  41711544  43479604  45556957  37737212  19909786
2014             18962736  44404208  38399690  40941085  44118256  18292243
2015             11066301  21666286  19710072  19289168  24408688   3796291


In [34]:
cumulative_triangle=triangle.cumsum(axis=1)    #cumulative summing across the columns instead of rows
print(cumulative_triangle)
df_observed=df[df["Development_Year"]<=2015]   #considering data only upto 2015 and ignoring values after 2015
df_observed.head()

Development_Lag         0         1          2          3          4  \
Origin_Year                                                            
2010             21163791  67741765  115129586  157797376  206668455   
2011             18474512  61092132  105344557  149948598  191934009   
2012             20757711  60469563  107386519  151909551  202018916   
2013             23638975  65350519  108830123  154387080  192124292   
2014             18962736  63366944  101766634  142707719  186825975   
2015             11066301  32732587   52442659   71731827   96140515   

Development_Lag          5  
Origin_Year                 
2010             224468337  
2011             219402726  
2012             222855751  
2013             212034078  
2014             205118218  
2015              99936806  


,Id,Ins_Age,BMI,Medical_History_1,Response,Issue_Date,Claim_Flag,delay_days,Payment_Date,Origin_Year,Development_Year,Development_Lag,Sum_Assured,Claim_Amount
1,5,0.059701,0.272288,5.0,4,2012-12-04,0,819,2015-03-03,2012,2015,3,445968,0
2,6,0.029851,0.428780,10.0,8,2010-04-03,0,818,2012-06-29,2010,2012,2,313457,0
3,7,0.164179,0.352438,0.0,8,2011-09-18,0,1197,2014-12-28,2011,2014,3,387053,0
4,8,0.417910,0.424046,4.0,8,2011-09-20,0,501,2013-02-02,2011,2013,2,258678,0
5,10,0.507463,0.364887,6.0,8,2011-10-11,0,1143,2014-11-27,2011,2014,3,163320,0


In [35]:
incremental_triangle=df_observed.pivot_table(index="Origin_Year",values="Claim_Amount",columns="Development_Lag",aggfunc="sum")   #creating a new chainladder triangle from the filtered data which does not contain values beyond 2015
print(incremental_triangle)

Development_Lag           0           1           2           3           4  \
Origin_Year                                                                   
2010             21163791.0  46577974.0  47387821.0  42667790.0  48871079.0   
2011             18474512.0  42617620.0  44252425.0  44604041.0  41985411.0   
2012             20757711.0  39711852.0  46916956.0  44523032.0         NaN   
2013             23638975.0  41711544.0  43479604.0         NaN         NaN   
2014             18962736.0  44404208.0         NaN         NaN         NaN   
2015             11066301.0         NaN         NaN         NaN         NaN   

Development_Lag           5  
Origin_Year                  
2010             17799882.0  
2011                    NaN  
2012                    NaN  
2013                    NaN  
2014                    NaN  
2015                    NaN  


In [36]:
cumulative_triangle=incremental_triangle.cumsum(axis=1)    #horizontal summation (columns) instead of vertical (rows)
print(cumulative_triangle)


Development_Lag           0           1            2            3  \
Origin_Year                                                         
2010             21163791.0  67741765.0  115129586.0  157797376.0   
2011             18474512.0  61092132.0  105344557.0  149948598.0   
2012             20757711.0  60469563.0  107386519.0  151909551.0   
2013             23638975.0  65350519.0  108830123.0          NaN   
2014             18962736.0  63366944.0          NaN          NaN   
2015             11066301.0         NaN          NaN          NaN   

Development_Lag            4            5  
Origin_Year                                
2010             206668455.0  224468337.0  
2011             191934009.0          NaN  
2012                     NaN          NaN  
2013                     NaN          NaN  
2014                     NaN          NaN  
2015                     NaN          NaN  


In [37]:
link_ratios=cumulative_triangle/cumulative_triangle.shift(axis=1)   #calculating link ratio by dividing lag1/lag0, lag2/lag1 and so on
print(round(link_ratios,3))

Development_Lag   0      1      2      3     4      5
Origin_Year                                          
2010            NaN  3.201  1.700  1.371  1.31  1.086
2011            NaN  3.307  1.724  1.423  1.28    NaN
2012            NaN  2.913  1.776  1.415   NaN    NaN
2013            NaN  2.765  1.665    NaN   NaN    NaN
2014            NaN  3.342    NaN    NaN   NaN    NaN
2015            NaN    NaN    NaN    NaN   NaN    NaN


In [38]:
ldf=link_ratios.mean()    #average of link ratios for every origin year
print(ldf)

Development_Lag
0         NaN
1    3.105392
2    1.716275
3    1.402874
4    1.294853
5    1.086128
dtype: float64


In [39]:
cdf_2015=ldf.prod()   #calculating the total link ratio across all years
print(cdf_2015)

10.515342293279685


In [40]:
current_paid_2015=cumulative_triangle.loc[2015,0]   #locating amount paid in the latest origin year
ultimate_2015=current_paid_2015*cdf_2015   #anticipating the amount to be paid in the ultimate year in the triangle
print(f"Current Paid: ${current_paid_2015:,.0f}")
print(f"Projected Ultimate: ${ultimate_2015:,.0f}")

Current Paid: $11,066,301
Projected Ultimate: $116,365,943


In [41]:
reserve_2015=ultimate_2015 - current_paid_2015   #amount to be kept as reserve, calculated as anticipated amount-amount paid in current year
print(f"Projected Ultimate:${ultimate_2015:,.0f}")
print(f"Already Paid:      -${current_paid_2015:,.0f}")
print(f"-------------------------------------")
print(f"IBNR Reserve Required:${reserve_2015:,.0f}")

Projected Ultimate:$116,365,943
Already Paid:      -$11,066,301
-------------------------------------
IBNR Reserve Required:$105,299,642


In [42]:
ldf=link_ratios.mean()   #calculating the average link ratio
ldf_reversed=ldf.dropna().iloc[::-1]   #reversing the ldf and dropping the null values 
cdf=ldf_reversed.cumprod().sort_index()   #calculating the cumulative lag 
print(cdf)

Development_Lag
1    10.515342
2     3.386156
3     1.972969
4     1.406376
5     1.086128
dtype: float64


In [43]:
summary=pd.DataFrame(index=cumulative_triangle.index)   #assigning development lag to the approriate year
summary["Lag"]=2015 - summary.index
print(summary)

             Lag
Origin_Year     
2010           5
2011           4
2012           3
2013           2
2014           1
2015           0


In [44]:
summary["CDF"]=summary["Lag"].map(lambda x:cdf.get(x+1,1))   #assigning the correct cdf of link ratio to the correct origin year as per summary table and cdf values
print(summary)
print(cumulative_triangle)

             Lag        CDF
Origin_Year                
2010           5   1.000000
2011           4   1.086128
2012           3   1.406376
2013           2   1.972969
2014           1   3.386156
2015           0  10.515342
Development_Lag           0           1            2            3  \
Origin_Year                                                         
2010             21163791.0  67741765.0  115129586.0  157797376.0   
2011             18474512.0  61092132.0  105344557.0  149948598.0   
2012             20757711.0  60469563.0  107386519.0  151909551.0   
2013             23638975.0  65350519.0  108830123.0          NaN   
2014             18962736.0  63366944.0          NaN          NaN   
2015             11066301.0         NaN          NaN          NaN   

Development_Lag            4            5  
Origin_Year                                
2010             206668455.0  224468337.0  
2011             191934009.0          NaN  
2012                     NaN          NaN  
201

In [45]:
summary["current_paid"]=cumulative_triangle.max(axis=1)   #adding the maximum values in the cumulative_triangle into the summary table for ease of calculation
print(summary)

             Lag        CDF  current_paid
Origin_Year                              
2010           5   1.000000   224468337.0
2011           4   1.086128   191934009.0
2012           3   1.406376   151909551.0
2013           2   1.972969   108830123.0
2014           1   3.386156    63366944.0
2015           0  10.515342    11066301.0


In [46]:
summary["Ultimate"]=summary["current_paid"]*summary["CDF"]   #calculating the ultimate column 
summary["Reserve"]=summary["Ultimate"] - summary["current_paid"]   #calculating the reserve column
pd.options.display.float_format='{:,.0f}'.format
print(summary)
print("-" * 40)
print(f"Total IBNR Reserve required: ${summary["Reserve"].sum():,.0f}")
summary.to_csv("final_reserves.csv")   #converting output to csv file

             Lag  CDF  current_paid    Ultimate     Reserve
Origin_Year                                                
2010           5    1   224,468,337 224,468,337           0
2011           4    1   191,934,009 208,464,847  16,530,838
2012           3    1   151,909,551 213,641,947  61,732,396
2013           2    2   108,830,123 214,718,438 105,888,315
2014           1    3    63,366,944 214,570,366 151,203,422
2015           0   11    11,066,301 116,365,943 105,299,642
----------------------------------------
Total IBNR Reserve required: $440,654,613


In [47]:
import sqlite3   #importing sql file to work in python
conn=sqlite3.connect("insurance_proj.db")   #saving new file named insurance_proj.db
df.to_sql("policies",conn,if_exists="replace",index=False)   #transferring df dataset to sqlite3
print(df)

          Id  Ins_Age  BMI  Medical_History_1  Response Issue_Date  \
0          2        1    0                  4         8 2012-02-03   
1          5        0    0                  5         4 2012-12-04   
2          6        0    0                 10         8 2010-04-03   
3          7        0    0                  0         8 2011-09-18   
4          8        0    0                  4         8 2011-09-20   
...      ...      ...  ...                ...       ...        ...   
59376  79142        0    1                  0         4 2015-05-16   
59377  79143        0    1                 24         7 2010-03-03   
59378  79144        0    0                  4         8 2010-04-10   
59379  79145        1    0                  0         8 2012-09-04   
59380  79146        0    1                  4         7 2013-02-02   

       Claim_Flag  delay_days Payment_Date  Origin_Year  Development_Year  \
0               0        1598   2016-06-19         2012              2016   
1    

In [48]:
query="""
SELECT Ins_Age, BMI, Medical_History_1,
        COUNT(*) as Exposure,  --Counts the total people in this group
        SUM(Claim_Flag) AS Claims  --Sums the deaths(1s) in this group
FROM policies
GROUP BY Ins_Age, BMI, Medical_History_1
"""
model_data=pd.read_sql_query(query,conn)

print("Aggregation complete!")
print(f"Original Rows: {len(df):,}")
print(f"Grouped Rows: {len(model_data):,}")
print("-"*40)
print(model_data.head())

Aggregation complete!
Original Rows: 59,381
Grouped Rows: 53,834
----------------------------------------
   Ins_Age  BMI  Medical_History_1  Exposure  Claims
0        0    0                  4         1       0
1        0    0                 12         1       0
2        0    0                  1         1       0
3        0    0                 12         1       0
4        0    0                 33         1       0


In [49]:
model_data.to_csv("pricing_model_data.csv",index=False)

In [50]:
import numpy as np
risk_probs=0.01+df["Ins_Age"]*0.15   #creating probability based on age instead of 50-50 binomial simulation
                                     #Base risk 1% + up to 15% more based on age
df["Claim_Flag"]=np.random.binomial(n=1,p=risk_probs)   #calculating new risk factor based on risk_pro
df.to_csv("final_data.csv",index=False)   
conn=sqlite3.connect("portfolio.db")
df.to_sql("policies",conn,if_exists="replace",index=False)



59381

In [51]:
query="""
SELECT Ins_Age,BMI,Medical_History_1,
    COUNT(*) AS Exposure,
    Sum(Claim_Flag) AS Claims
from policies
GROUP BY Ins_Age,BMI,Medical_History_1
"""

model_data=pd.read_sql_query(query,conn)
model_data.to_csv("pricing_model_data.csv",index=False)